Refernce: https://www.beelinereader.com/

In [ ]:
!pip install torch transformers nltk

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import nltk
from transformers import BertTokenizer, BertModel
import torch

nltk.download('punkt')

def preprocess_text(text):
    # Tokenize text
    sentences = nltk.sent_tokenize(text)
    words = [nltk.word_tokenize(sentence) for sentence in sentences]
    return words


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
class FeatureExtractor:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def extract_features(self, sentences):
        features = []
        for sentence in sentences:
            inputs = self.tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
            outputs = self.model(**inputs)
            features.append(outputs.last_hidden_state)
        return features


In [ ]:
import torch.nn as nn

class ColorTransitionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ColorTransitionModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        return out


In [ ]:
def train_model(model, data_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for features in data_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


In [ ]:
def color_mapping(predictions):
    # Simple gradient from blue to red
    colors = []
    for pred in predictions:
        r = int(255 * pred)
        g = 0
        b = int(255 * (1 - pred))
        colors.append(f'rgb({r},{g},{b})')
    return colors


In [ ]:
def render_text(sentences, colors):
    html = ""
    for sentence, sentence_colors in zip(sentences, colors):
        for word, color in zip(sentence, sentence_colors):
            html += f'<span style="color:{color}">{word} </span>'
        html += '<br>'
    return html


In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Sample text
text = "Deep learning models are powerful and versatile. They can be used for a variety of tasks."

# Preprocess text
words = preprocess_text(text)

# Extract features
feature_extractor = FeatureExtractor()
features = feature_extractor.extract_features(words)

# Convert to TensorDataset and DataLoader for training
features_tensor = torch.cat(features)
dataset = TensorDataset(features_tensor)
data_loader = DataLoader(dataset, batch_size=2)

# Initialize model, criterion, and optimizer
input_dim = features_tensor.size(-1)
hidden_dim = 128
output_dim = 1
model = ColorTransitionModel(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model (placeholder, no real labels)
train_model(model, data_loader, criterion, optimizer, epochs=1)

# Predict color transitions
model.eval()
with torch.no_grad():
    predictions = model(features_tensor).squeeze().tolist()

# Map predictions to colors
colors = color_mapping(predictions)

# Render text
html_output = render_text(words, colors)
print(html_output)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

AttributeError: 'list' object has no attribute 'dim'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import nltk
from transformers import BertTokenizer, BertModel

# Sample text
text = "Deep learning models are powerful and versatile. They can be used for a variety of tasks."

# Preprocess text
nltk.download('punkt')

def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    words = [nltk.word_tokenize(sentence) for sentence in sentences]
    return words

words = preprocess_text(text)

class FeatureExtractor:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def extract_features(self, sentences):
        features = []
        for sentence in sentences:
            inputs = self.tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
            outputs = self.model(**inputs)
            features.append(outputs.last_hidden_state)
        return features

# Extract features
feature_extractor = FeatureExtractor()
features = feature_extractor.extract_features([' '.join(sentence) for sentence in words])

# Convert to TensorDataset and DataLoader for training
features_tensor = torch.cat(features, dim=1).squeeze(0)
dataset = TensorDataset(features_tensor)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Define ColorTransitionModel
class ColorTransitionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ColorTransitionModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x.unsqueeze(0))  # Adding batch dimension
        out = self.fc(lstm_out.squeeze(0))       # Removing batch dimension
        return out

# Initialize model, criterion, and optimizer
input_dim = features_tensor.size(-1)
hidden_dim = 128
output_dim = 1
model = ColorTransitionModel(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (placeholder, no real labels)
def train_model(model, data_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for batch in data_loader:
            optimizer.zero_grad()
            features = batch[0]
            outputs = model(features)
            loss = criterion(outputs, torch.zeros_like(outputs))  # Dummy target
            loss.backward(retain_graph=True)
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train_model(model, data_loader, criterion, optimizer, epochs=1)

# Predict color transitions
model.eval()
with torch.no_grad():
    predictions = model(features_tensor).squeeze().tolist()

# Map predictions to colors with inverted colors for darker shades
def color_mapping(predictions):
    colors = []
    for pred in predictions:
        # Normalizing the prediction to be between 0 and 1
        normalized_pred = (pred - min(predictions)) / (max(predictions) - min(predictions))
        # Mapping normalized predictions to a range of darker colors
        if normalized_pred < 0.33:
            # Dark Red to Dark Yellow
            r = int(255 * (1 - (normalized_pred / 0.33)))
            g = int(100 * (1 - (normalized_pred / 0.33)))
            b = int(0)
        elif normalized_pred < 0.66:
            # Dark Yellow to Dark Green
            r = int(100 * (1 - ((normalized_pred - 0.33) / 0.33)))
            g = int(255 * (1 - ((normalized_pred - 0.33) / 0.33)))
            b = int(0)
        else:
            # Dark Green to Dark Blue
            r = int(0)
            g = int(100 * (1 - ((normalized_pred - 0.66) / 0.34)))
            b = int(255 * (1 - ((normalized_pred - 0.66) / 0.34)))
        colors.append(f'rgb({r},{g},{b})')
    return colors

colors = color_mapping(predictions)

# Render text
def render_text(sentences, colors):
    html = ""
    color_index = 0
    for sentence in sentences:
        for word in sentence:
            html += f'<span style="color:{colors[color_index]}">{word} </span>'
            color_index += 1
        html += '<br>'
    return html

html_output = render_text(words, colors)
print(html_output)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1, Loss: 0.014839516952633858
<span style="color:rgb(92,236,0)">Deep </span><span style="color:rgb(255,100,0)">learning </span><span style="color:rgb(86,219,0)">models </span><span style="color:rgb(0,85,217)">are </span><span style="color:rgb(0,85,217)">powerful </span><span style="color:rgb(34,89,0)">and </span><span style="color:rgb(0,84,214)">versatile </span><span style="color:rgb(41,105,0)">. </span><br><span style="color:rgb(20,53,0)">They </span><span style="color:rgb(27,71,0)">can </span><span style="color:rgb(61,155,0)">be </span><span style="color:rgb(73,188,0)">used </span><span style="color:rgb(0,65,166)">for </span><span style="color:rgb(0,23,60)">a </span><span style="color:rgb(0,55,140)">variety </span><span style="color:rgb(90,231,0)">of </span><span style="color:rgb(0,0,0)">tasks </span><span style="color:rgb(0,83,213)">. </span><br>


In [ ]:
e

In [4]:
   from IPython.display import HTML

   html_output = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Color Gradient Text</title>
    <style>
        .gradient-text-1 {
            background: linear-gradient(to right, blue, blue, black);
            -webkit-background-clip: text;
            color: transparent;
        }
        .gradient-text-2 {
            background: linear-gradient(to right, blue, black, blue);
            -webkit-background-clip: text;
            color: transparent;
        }
        .gradient-text-3 {
            background: linear-gradient(to right, red, brown, black);
            -webkit-background-clip: text;
            color: transparent;
        }
    </style>
</head>
<body>
    <p>
        <span class="gradient-text-1">Lorem Ipsum is simply dummy text of the printing and typesetting industry.</span>
        <span class="gradient-text-2"> Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.</span>
        <span class="gradient-text-3"> It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged.</span>
        <span class="gradient-text-2"> It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.</span>
    </p>
</body>
</html>
   """

   display(HTML(html_output))